## Setup

In [1]:
# Install SymPy if it isn't installed already
# Ensure that PyCall uses the Conda.jl package's Miniconda distribution within Julia
# Otherwise the sympy Python module won't be imported properly.
ENV["PYTHON"]=""

if !isdir(Pkg.dir("PyCall"))
    Pkg.add("PyCall")
end

if !isdir(Pkg.dir("SymPy"))
    # Add SymPy
    Pkg.build("PyCall")
    Pkg.add("SymPy")
end

In [ ]:
using RigidBodyDynamics
using StaticArrays
using SymPy

INFO: Recompiling stale cache file /Users/twan/code/julia/RigidBodyDynamics/lib/v0.5/PyCall.ji for module PyCall.
INFO: Recompiling stale cache file /Users/twan/code/julia/RigidBodyDynamics/lib/v0.5/SymPy.ji for module SymPy.
INFO: Installing sympy via the Conda sympy package...


Fetching package metadata .........
Solving package specifications: .

Package plan for installation in environment /Users/twan/code/julia/RigidBodyDynamics/v0.5/Conda/deps/usr:

The following NEW packages will be INSTALLED:

    mpmath: 0.19-py27_1
    sympy:  1.0-py27_0 

mpmath-0.19-py 100% |###############################| ETA:  0:00:00  27.39 MB/smpmath-0.19-py   3% |#                              | ETA:  0:00:00  13.39 MB/smpmath-0.19-py   7% |##                             | ETA:  0:00:00  15.09 MB/smpmath-0.19-py  10% |###                            | ETA:  0:00:00  17.91 MB/smpmath-0.19-py  14% |####                           | ETA:  0:00:00  19.17 MB/smpmath-0.19-py  18% |#####                          | ETA:  0:00:00  20.68 MB/smpmath-0.19-py  21% |######                         | ETA:  0:00:00  21.90 MB/smpmath-0.19-py  25% |#######                        | ETA:  0:00:00  20.89 MB/smpmath-0.19-py  29% |#########                      | ETA:  0:00:00  21.74 MB/smpmath-0.19-py

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [ ]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitationalAcceleration = @syms g real = true
params = [inertias..., lengths..., gravitationalAcceleration...]

## Create double pendulum `Mechanism`

A `Mechanism` contains the joint layout and inertia parameters, but no state information.

In [ ]:
T = Sym # the 'scalar type' of the Mechanism we'll construct
axis = SVector(zero(T), one(T), zero(T)) # axis of rotation for each of the joints
doublePendulum = Mechanism(RigidBody{T}("world"); gravity = SVector(0, 0, g))
world = root_body(doublePendulum) # the fixed 'world' rigid body

# Attach the first (upper) link to the world via a revolute joint named 'shoulder'
inertia1 = SpatialInertia(CartesianFrame3D("upper_link"), I_1 * axis * axis', SVector(0, 0, c_1), m_1)
body1 = RigidBody(inertia1)
joint1 = Joint("shoulder", Revolute(axis))
joint1ToWorld = Transform3D{T}(joint1.frameBefore, default_frame(world))
attach!(doublePendulum, world, joint1, joint1ToWorld, body1)

# Attach the second (lower) link to the world via a revolute joint named 'elbow'
inertia2 = SpatialInertia(CartesianFrame3D("lower_link"), I_2 * axis * axis', SVector(0, 0, c_2), m_2)
body2 = RigidBody(inertia2)
joint2 = Joint("elbow", Revolute(axis))
joint2ToBody1 = Transform3D(joint2.frameBefore, default_frame(body1), SVector(0, 0, l_1))
attach!(doublePendulum, body1, joint2, joint2ToBody1, body2)

## Create `MechanismState` associated with the double pendulum `Mechanism`

A `MechanismState` stores all state-dependent information associated with a `Mechanism`.

In [ ]:
x = MechanismState(T, doublePendulum);

In [ ]:
# Set the joint configuration vector of the MechanismState to a new vector of symbolic variables
q = configuration_vector(x)[:] = [symbols("q_$i", real = true) for i = 1 : num_positions(x)]

In [ ]:
# Set the joint velocity vector of the MechanismState to a new vector of symbolic variables
v = velocity_vector(x)[:] = [symbols("v_$i", real = true) for i = 1 : num_positions(x)]

## Compute dynamical quantities in symbolic form

In [ ]:
# Mass matrix
M = mass_matrix(x)
function simplify!(a::Array{SymPy.Sym})
    for i in eachindex(a)
        a[i] = simplify(a[i])
    end
    a
end
simplify!(M.data) # Note: M is a Symmetric matrix type; need to simplify the underlying data
full(M) # convert to full form so that it is pretty-printed (minor bug in SymPy.jl)

In [ ]:
# Kinetic energy
simplify(kinetic_energy(x))

In [ ]:
# Potential energy
simplify(potential_energy(x))